In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
# Path to the secret key file
secret_file_path = '/content/drive/MyDrive/AIUSNEPAL/secret.txt'

# Read the secret key from the file
with open(secret_file_path, 'r') as f:
    secret_key = f.read().strip()

key = secret_key.split("=")[1]

In [55]:
import os
os.environ["OPENAI_API_KEY"] =  key
os.environ["PINECONE_API_KEY"]="pcsk_4M7zx2_GnYvdD5RYsxjd7iavo6ojFz1Yx5Kvq9Ts3Fp2G8CPjNo9kKiSauyhdm5K57H1QG"

In [4]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [5]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [14]:
def get_pdf_file(data):
    loader= DirectoryLoader(data,
                        glob="*.pdf",
                        loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [16]:
pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.5 MB/s eta 0:00:00


In [17]:
extracted_data=get_pdf_file(data='/content/drive/MyDrive/AIUSNEPAL/')

chunking


In [19]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [20]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [33]:
from langchain.embeddings import OpenAIEmbeddings
def download_embeddings():
  embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=os.environ["OPENAI_API_KEY"])
  return embeddings


In [34]:
embeddings=download_embeddings()

In [26]:
pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.6 MB/s eta 0:00:00


In [35]:
query_result=embeddings.embed_query("Hello")

pcsk_4M7zx2_GnYvdD5RYsxjd7iavo6ojFz1Yx5Kvq9Ts3Fp2G8CPjNo9kKiSauyhdm5K57H1QG

In [42]:
pip install pinecone-client


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: pinecone-plugin-inference
    Found existing installation: pinecone-plugin-inference 3.1.0
    Uninstalling pinecone-plugin-inference-3.1.0:
      Successfully uninstalled pinecone-plugin-inference-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pinecone 5.4.2 requires pinecone-plugin-inference<4.0.0,>=2.0.0, but you have pinecone-plugin-inference 1.1.0 which is incompatible.


In [61]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="pcsk_4M7zx2_GnYvdD5RYsxjd7iavo6ojFz1Yx5Kvq9Ts3Fp2G8CPjNo9kKiSauyhdm5K57H1QG")


environment = "us-east-1-aws"  # Replace with your Pinecone environment

index_name = "mediexpert"

pc.create_index(
    name=index_name,
    dimension=1536, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)


In [51]:
pip install langchain pinecone-client


In [53]:
pip install langchain-pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.4 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.10
    Uninstalling aiohttp-3.11.10:
      Successfully uninstalled aiohttp-3.11.10


In [62]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [64]:
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [66]:
docsearch

In [68]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [69]:
retrieved_docs = retriever.invoke("What is Acne?")
retrieved_docs

[Document(id='013e7e9a-bf38-466a-98ab-c39ff82a6cfb', metadata={'page': 241.0, 'source': '/content/drive/MyDrive/AIUSNEPAL/Medical_book.pdf'}, page_content='KEY TERMS\nAcne—A skin condition in which raised bumps,\npimples, and cysts form on the face, neck, shoul-\nders and upper back.\nBacteria—Tiny, one-celled forms of life that cause\nmany diseases and infections.\nBowel—The intestine; a tube-like structure that\nextends from the stomach to the anus. Some diges-\ntive processes are carried out in the bowel before\nfood passes out of the body as waste.\nCyst—An abnormal sac or enclosed cavity in the\nbody, filled with liquid or partially solid material.'),
 Document(id='9a526434-fab6-4ccb-8ad3-91701a35de21', metadata={'page': 38.0, 'source': '/content/drive/MyDrive/AIUSNEPAL/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.

In [71]:
pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [72]:

from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [73]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [74]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [75]:

response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])



Acromegaly and gigantism are disorders caused by the abnormal release of a chemical from the pituitary gland in the brain, resulting in increased growth in bone and soft tissue and other disturbances in the body. This abnormality can occur before or after bone growth stops, with acromegaly occurring after bone growth stops. It is a rare disorder that affects both men and women, with symptoms often being identified in middle age due to the gradual onset of symptoms.


In [77]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])



I don't know.
